In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

In [2]:
#data = body of post request will all values of first and third picker + 1st value of 2nd picker (use postman)
with(open('post_request_body.txt', "rb")) as f:
    body = [tuple(x.decode().strip().split(':')) for x in f.readlines()]

In [3]:
#find index of the variable in the body
key = 'ctl00$ContentPlaceHolderMain$VariableSelector1$VariableSelector1$VariableSelectorValueSelectRepeater$ctl02$VariableValueSelect$VariableValueSelect$ValuesListBox'

for idx, item in enumerate(body): 
    if item[0] == key: 
        break
idx

2457

In [4]:
#find all options for second picker
URL = "https://www.pxweb.bfs.admin.ch/Selection.aspx?px_language=fr&px_db=px-x-1703030000_101&px_tableid=px-x-1703030000_101/px-x-1703030000_101.px&px_type=PX"
soup = BeautifulSoup(requests.get(URL).text, 'html.parser')
options = [o.attrs['value'] for o in soup.select('#ctl00_ContentPlaceHolderMain_VariableSelector1_VariableSelector1_VariableSelectorValueSelectRepeater_ctl02_VariableValueSelect_VariableValueSelect_ValuesListBox option')]
len(options)

312

In [5]:
#Note URL_post != URL
URL_post = 'https://www.pxweb.bfs.admin.ch/Selection.aspx?px_language=fr&px_db=px-x-1703030000_101&px_tableid=px-x-1703030000_101%2fpx-x-1703030000_101.px&px_type=PX'

df = pd.DataFrame()

for o in tqdm_notebook(options):
    
    #Change the value of the variable in the body of the post request
    body[idx] = (key, o)
    
    #Make the post request
    r = requests.post(URL_post, data=body)
    
    #Build the temporary dataframe corresponding to one votation
    df_tmp = pd.read_html(r.text, thousands=' ', decimal=',', na_values=['...'])[0]
    df_tmp.columns = df_tmp.columns.droplevel()
    df_tmp.columns = ['Commune', 'Votation'] + df_tmp.columns[1:-1].tolist()
    
    #Concat with the temporary dataframe with the dataframe containing all the votations
    df = pd.concat([df, df_tmp])
    
df = df.sort_index()
df = df.set_index(['Commune', 'Votation'])

df

A Jupyter Widget

Electeurs inscrits  \
Commune                           Votation                                                                 
Suisse                            21.05.2017 Loi sur l'énergie                                 5356575.0   
                                  10.03.1996 Places de stationnement près des gares            4599427.0   
                                  09.06.1996 Article sur l'agriculture                         4602577.0   
                                  09.06.1996 Organisation du gouvernement et de l...           4602577.0   
                                  09.06.1985 Part cantonal aux droits de timbre                4144312.0   
                                  27.11.2016 Initiative pour la sortie du nucléaire            5336711.0   
                                  01.12.1996 Contre l'immigration clandestine                  4612266.0   
                                  01.12.1996 Loi sur le travail                                4612266.0   
                                  08.06.1997 Négociations d'adhésion à l'UE: que ...           4614662.0   
                                  08.06.1997 Initiative contre l'exportation de m...           4614662.0   
                                  08.06.1997 Suppression de la régale des poudres              4614662.0   
                                  28.09.1997 Financement de l'assurance-chômage                4620084.0   
                                  09.06.1985 Pour le droit à la vie                            4144312.0   
                                  28.09.1997 Initiative «Jeunesse sans drogue»                 4620084.0   
                                  07.06.1998 Objectif budgétaire 2001                          4629496.0   
                                  07.06.1998 Initiative pour la protection génétique           4629496.0   
                                  10.03.1996 Rachat d'eau-de-vie et d'appareils à...           4599427.0   
                                  07.06.1998 Initiative contre une police fouineuse            4629496.0   
                                  10.03.1996 Equipement personnel des militaires               4599427.0   
                                  10.03.1996 Article sur les langues                           4599427.0   
                                  12.06.1994 Casques bleus                                     4572713.0   
                                  25.09.1994 Suppression de la réduction du prix ...           4576512.0   
                                  25.09.1994 Loi contre le racisme                             4576512.0   
                                  04.12.1994 Loi sur l'assurance-maladie                       4580035.0   
                                  04.12.1994 Initiative «pour une saine assurance...           4580035.0   
                                  04.12.1994 Mesures de contrainte concernant les...           4580035.0   
                                  09.06.1985 Suppression de l'aide aux producteur...           4144312.0   
                                  12.03.1995 Article sur l'agriculture                         4583930.0   
                                  12.03.1995 Economie laitière                                 4583930.0   
                                  12.03.1995 Loi sur l'agriculture                             4583930.0   
...                                                                                                  ...   
...... TI-voto per corrispondenza 06.06.1993 40 places d'armes ça suffit                             NaN   
                                  07.03.1993 Abolition de la vivisection                             NaN   
                                  07.03.1993 Autorisation des maisons de jeu                         NaN   
                                  07.03.1993 Droits d'entrée sur les carburants                      NaN   
                                  06.12.1992 Adhésion à l'EEE                             

In [7]:
df.to_pickle('votations.pkl')

In [ ]:
# DO NOT EXECUTE 
# MIGHT STILL BE USEFUL, BE KIND AND DO NOT DELETE

# GET, params saving
r = requests.get(URL).text
soup = bs(r, 'html.parser')
event_validation = soup.find(id="__EVENTVALIDATION").attrs['value']
view_state = soup.find(id="__VIEWSTATE").attrs['value']
view_stat_generator = soup.find(id="__VIEWSTATEGENERATOR").attrs['value']

d['__EVENTVALIDATION'] = event_validation
d['__VIEWSTATE'] = view_state
d['__VIEWSTATEGENERATOR'] = view_stat_generator